In [1]:
print("Importing dependencies..")
%load_ext autoreload
%autoreload 2
import scipy
import numpy as np
from scipy.linalg import lstsq
from scipy.linalg import norm 
import pandas as pd
import os
# from util import RegressionGame
# from util_sparse import getShapleyProjection
import os
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm
# import shap
# import llm_helper

if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.exists('__dcache__'):
    os.makedirs('__dcache__')

# N = 6000
# print("Explanation count: %s" % N)
# k = 100
# print("SHAP sample count: %s" % k)
seq_len = 50
print("Seq Length: %s" % seq_len)

# Setup
np.random.seed(1)
# model = llm_helper.get_model()

Importing dependencies..
Seq Length: 50


# Experiment 2

In [2]:
cuda = True

from transformers import GPT2LMHeadModel, GPT2TokenizerFast
if cuda:
    model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
else:
    model = GPT2LMHeadModel.from_pretrained('gpt2')

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2', use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
test = pd.read_pickle('../mwe_tagger/fsample.pkl')
test['length'] = test['sentence'].str.split().str.len()
# X = tokenizer(test['sentence'].to_list(), padding=True,  truncation=True, max_length=seq_len, return_tensors ='pt').input_ids
# print(X.shape)

In [3]:
# X = X.cuda()

In [4]:
# from joblib import Parallel, delayed
# num_cores = 10

def get_prediction_softmax(model, X, token_next):
    if cuda:
        return torch.softmax(model(X).logits[token_next:], dim=1)
    else:
        return scipy.special.softmax(model(X).logits[token_next:].detach().numpy(), axis=1)

def interaction_value_di(model, X, tokens):
    token1, token2 = tokens
    
    token_next = max(token1, token2) + 1
    AB = get_prediction_softmax(model, X, token_next)
    
    X_t1 = X.clone()
    X_t1[token1] = tokenizer.unk_token_id
    A = get_prediction_softmax(model, X_t1, token_next)
    
    X_t2 = X.clone()
    X_t2[token2] = tokenizer.unk_token_id
    B = get_prediction_softmax(model, X_t2, token_next)

    X_t12 = X.clone()
    X_t12[token2] = tokenizer.unk_token_id
    X_t12[token1] = tokenizer.unk_token_id
    phi = get_prediction_softmax(model, X_t12, token_next)
    
    val = AB - A - B + phi
    if cuda:
        val = torch.linalg.norm(val, dim=1).cpu()
    else:
        val = np.linalg.norm(val, axis=1)
    
    del X_t1, X_t2, X_t12,  AB, A, B, phi
    if cuda:
        return val.detach(), token_next
    else:
        return val, token_next

def mwe_distance_interaction(encoded_row, row, col, row_number):
    iv_mwe = []
    # col = 'weak_mwe' | 'strong_mwe'
    
    mwes = row[col]
    for mwe in mwes:

        for i in range(len(mwe)):
            for j in range(len(mwe)):
                if i > j:
                    if len([x for x in mwe if x >= seq_len]) > 0:
                        continue
                    
                    iv = interaction_value_di(model, encoded_row, [mwe[i], mwe[j]])
                    iv_mwe.append([iv, abs((mwe[i]-mwe[j])),mwe, row_number, i, j])
    # print(torch.cuda.memory_allocated()/(1024*1024*1024))
    return iv_mwe
weak_mwe_distance = []
strong_mwe_distance = []


for row_number, row in tqdm(test.iterrows()):
    # if row_number%100==0:
        # print(row_number, len(test), f'{row_number*100/len(test)}%')
    abc =  tokenizer(row['sentence'], padding=False,  truncation=True, max_length=seq_len, return_tensors ='pt').input_ids[0]
    if cuda:
        abc = abc.cuda()

    
    # print("Start",torch.cuda.memory_allocated()/(1024*1024*1024))
    weak_mwe_distance.extend(mwe_distance_interaction(abc, row, 'weak_mwe', row_number))
    strong_mwe_distance.extend(mwe_distance_interaction(abc, row, 'strong_mwe', row_number))
    
    # print(torch.cuda.memory_allocated()/(1024*1024*1024))
    del abc



10265it [26:16,  6.51it/s]


In [5]:
# torch.cuda.memory_snapshot()

[{'device': 0,
  'address': 42991616000,
  'total_size': 20971520,
  'allocated_size': 19660800,
  'active_size': 19660800,
  'requested_size': 19660800,
  'stream': 0,
  'segment_type': 'large',
  'blocks': [{'size': 3145728,
    'requested_size': 3145728,
    'state': 'active_allocated'},
   {'size': 7077888, 'requested_size': 7077888, 'state': 'active_allocated'},
   {'size': 2359296, 'requested_size': 2359296, 'state': 'active_allocated'},
   {'size': 7077888, 'requested_size': 7077888, 'state': 'active_allocated'},
   {'size': 1310720, 'requested_size': 1206168, 'state': 'inactive'}]},
 {'device': 0,
  'address': 43012587520,
  'total_size': 2097152,
  'allocated_size': 1540096,
  'active_size': 1540096,
  'requested_size': 1534000,
  'stream': 0,
  'segment_type': 'small',
  'blocks': [{'size': 3072,
    'requested_size': 3072,
    'state': 'active_allocated'},
   {'size': 3072, 'requested_size': 3072, 'state': 'active_allocated'},
   {'size': 9216, 'requested_size': 9216, 'state

In [6]:
pd.DataFrame(weak_mwe_distance, columns = ['I', 'posdis', 'ignore', 'row_number', 'first_token', 'second_token']).to_pickle('weak_mwe_distance_3d.pkl')
pd.DataFrame(strong_mwe_distance, columns = ['I', 'posdis', 'ignore', 'row_number', 'first_token', 'second_token']).to_pickle('strong_mwe_distance_3d.pkl')

In [7]:
pd.DataFrame(strong_mwe_distance, columns = ['I', 'posdis', 'ignore', 'row_number', 'first_token', 'second_token'])

,I,posdis,ignore,row_number,first_token,second_token
0,"([tensor(0.2753)], 40)",1,"[38, 39]",373,1,0
1,"([tensor(0.2149), tensor(0.0847), tensor(0.092...",1,"[12, 13]",418,1,0
2,"([tensor(0.0558), tensor(0.2328), tensor(0.130...",1,"[5, 6, 7]",462,1,0
3,"([tensor(0.1867), tensor(0.1216), tensor(0.149...",2,"[5, 6, 7]",462,2,0
4,"([tensor(0.3418), tensor(0.1314), tensor(0.195...",1,"[5, 6, 7]",462,2,1
...,...,...,...,...,...,...
324,"([tensor(0.6572), tensor(0.2784), tensor(0.260...",1,"[11, 12, 13, 14, 15]",10006,3,2
325,"([tensor(0.3913), tensor(0.0865), tensor(0.028...",4,"[11, 12, 13, 14, 15]",10006,4,0
326,"([tensor(0.3599), tensor(0.1236), tensor(0.045...",3,"[11, 12, 13, 14, 15]",10006,4,1
327,"([tensor(0.3050), tensor(0.2664), tensor(0.046...",2,"[11, 12, 13, 14, 15]",10006,4,2


In [ ]:
import os
import datetime
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
print("Fetching Average Distance...")


def calculate_interaction(encoded_row, row, row_number):
    interactions = []
    encoded_len = len(encoded_row)
    for j in range( min(seq_len, encoded_len)):
        for k in range(j+1, min(seq_len, encoded_len, j+14)):
            if j+k >= encoded_len:
                continue
            if encoded_row[j] == tokenizer.unk_token_id or encoded_row[k] == tokenizer.unk_token_id:
                continue
            
            iv = interaction_value_di(model, encoded_row, [j, k])
            interactions.append([iv, abs(k -j),row_number, [j,k]])
    return interactions

average_distance = []
start_time =  datetime.datetime.now()
# min_row = average_distance[-1][2]
# print(min_row)

for row_number, row in tqdm(test.iterrows()):
    # if i%9 == 0 or i<=100:
    # print(i, len(test), f'{i*100/len(test)}%', (datetime.datetime.now() - start_time).seconds)
    # start_time = datetime.datetime.now()
    abc =  tokenizer(row['sentence'], padding=False,  truncation=True, max_length=seq_len, return_tensors ='pt').input_ids[0]
    if cuda:
        abc = abc.cuda()

    
    average_distance.extend(calculate_interaction(abc, row, row_number))

# from joblib import Parallel, delayed
# 
# num_cores = 9
# for x in range(0, len(test), 99):
#     print(x, len(test), f'{100*x/len(test)}%')
#     fabc =  lambda x : tokenizer(x, padding=False,  truncation=True, max_length=seq_len, return_tensors ='pt').input_ids[0]
    
#     average_distance.extend(Parallel(n_jobs=num_cores)(
#         delayed(calculate_interaction)(fabc(row['sentence']), row, i) for i, row in tqdm(test[x:x+99].iterrows())))

    

Fetching Average Distance...


6618it [17:46:48, 10.71s/it]

In [ ]:
import pickle
pickle.dump(average_distance, open('average_distance_3d.pkl','wb'))

In [ ]:
pd.DataFrame(average_distance, columns = ['I', 'posdis', 'row_number', 'word_pair']).to_pickle('average_distance_3d.pkl')

### DATA

In [ ]:
test = pd.read_pickle('../mwe_tagger/fsample.pkl')
a = np.unique([y[-1]-y[0] for x in test['_'].to_list() for y in x], return_counts=True)
print({x:y for x,y in zip(a[0], a[1][::-1].cumsum()[::-1])})
print(np.unique([z for x in test['_'].to_list() for y in x for z in y], return_counts=True))

# EXPERIMENT 1

In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2', use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
test = pd.read_pickle('../mwe_tagger/fsample.pkl')
test['length'] = test['sentence'].str.split().str.len()
test = test[test['length'] > seq_len].copy().reset_index(drop=True)
X = tokenizer(test['sentence'].to_list(), padding=True,  truncation=True, max_length=seq_len, return_tensors ='pt').input_ids
print(X.shape)

In [ ]:
# from joblib import Parallel, delayed
# num_cores = 10

def get_prediction_softmax(model, X):
    return scipy.special.softmax(model(X).logits[-1].detach().numpy())

def interaction_value_di(model, X, tokens):
    token1, token2 = tokens
    AB = get_prediction_softmax(model, X)
    
    X_t1 = X.clone()
    X_t1[token1] = tokenizer.unk_token_id
    A = get_prediction_softmax(model, X_t1)
    
    X_t2 = X.clone()
    X_t2[token2] = tokenizer.unk_token_id
    B = get_prediction_softmax(model, X_t2)

    
    X_t12 = X.clone()
    X_t12[token2] = tokenizer.unk_token_id
    X_t12[token1] = tokenizer.unk_token_id
    phi = get_prediction_softmax(model, X_t12)
    
    val = AB - A - B + phi
    val = np.linalg.norm(val)
    return val

def mwe_distance_interaction(encoded_row, row, col):
    iv_mwe = []
    # col = 'weak_mwe' | 'strong_mwe'
    mwes = row[col]
    for mwe in mwes:

        for i in range(len(mwe)):
            for j in range(len(mwe)):
                if i > j:
                    if len([x for x in mwe if x >= seq_len]) > 0:
                        continue
                    iv = interaction_value_di(model, encoded_row, [mwe[i], mwe[j]])
                    iv_mwe.append([iv, abs((mwe[i]-mwe[j])),mwe])
    return iv_mwe
weak_mwe_distance = []
strong_mwe_distance = []

# weak_mwe_distance = Parallel(n_jobs=num_cores)(
#     delayed(mwe_distance_interaction)(X[i], row, 'weak_mwe') for i, row in test.iterrows())

# strong_mwe_distance = Parallel(n_jobs=num_cores)(
#     delayed(mwe_distance_interaction)(X[i], row, 'strong_mwe') for i, row in test.iterrows())


for i, row in test.iterrows():
    if i%100==0:
        print(i, len(test), f'{i*100/len(test)}%')
    weak_mwe_distance.extend(mwe_distance_interaction(X[i], row, 'weak_mwe'))
    strong_mwe_distance.extend(mwe_distance_interaction(X[i], row, 'strong_mwe'))

pd.DataFrame(weak_mwe_distance, columns = ['I', 'posdis', 'ignore']).to_pickle('weak_mwe_distance1.pkl')
pd.DataFrame(strong_mwe_distance, columns = ['I', 'posdis', 'ignore']).to_pickle('strong_mwe_distance1.pkl')

In [ ]:
import os
import datetime
os.environ["TOKENIZERS_PARALLELISM"] = "false"
print("Fetching Average Distance...")

from joblib import Parallel, delayed

def calculate_interaction(encoded_row, row):
    interactions = []
    for j in range( seq_len):
        for k in range(j+1, seq_len):

            if j+k >= seq_len:
                continue
            if encoded_row[j] == tokenizer.unk_token_id or encoded_row[k] == tokenizer.unk_token_id:
                continue
            
            iv = interaction_value_di(model, encoded_row, [j, k])
            interactions.append([iv, abs(k -j),[j,k]])
    return interactions

average_distance = []
# start_time =  datetime.datetime.now()
# for i, row in test.iterrows():
#     # if i%9 == 0 or i<=100:
#     print(i, len(test), f'{i*100/len(test)}%', (datetime.datetime.now() - start_time).seconds)
#     start_time = datetime.datetime.now()
#     average_distance.extend(calculate_interaction(X[i], row))

num_cores = 9
for x in range(0, len(test), 99):
    print(x, len(test), f'{100*x/len(test)}%')
    average_distance.extend(Parallel(n_jobs=num_cores)(
        delayed(calculate_interaction)(X[i], row) for i, row in tqdm(test[x:x+99].iterrows())))


In [ ]:
avg_dist = [item for sublist in average_distance for item in sublist]
pd.DataFrame(avg_dist, columns = ['I', 'posdis', 'ignore']).to_pickle('average_distance1.pkl')

In [ ]:

# avg_dist = [item for sublist in average_distance for item in sublist]

def plot_mean(avg_path='average_distance.pkl', weak_path='weak_mwe_distance.pkl', strong_path='strong_mwe_distance.pkl'):
    avg_dist = pd.read_pickle(avg_path)
    avg_df = pd.DataFrame(avg_dist, columns = ['I', 'posdis', 'ignore']).groupby('posdis')['I'].agg( ['mean', 'count', 'std']).rename(columns = {'mean':'avg_mean', 'count':'avg_count', 'std':'avg_std'})
    weak_mwe_distance = pd.read_pickle(weak_path)
    weak_mwe_df = pd.DataFrame(weak_mwe_distance, columns = ['I', 'posdis', 'ignore']).groupby('posdis')['I'].agg( ['mean', 'count', 'std']).rename(columns = {'mean':'weak_mean', 'count':'weak_count', 'std':'weak_std'})
    weak_mwe_df = weak_mwe_df.drop(0)
    strong_mwe_distance = pd.read_pickle(strong_path)
    strong_mwe_df = pd.DataFrame(strong_mwe_distance, columns = ['I', 'posdis', 'ignore']).groupby('posdis')['I'].agg( ['mean', 'count', 'std']).rename(columns = {'mean':'strong_mean', 'count':'strong_count', 'std':'strong_std'})
    abc = pd.concat([avg_df, weak_mwe_df, strong_mwe_df], axis=1)
    display(abc)
    abc = abc[abc['weak_count'] >=50]
    abc[['avg_mean', 'weak_mean']].plot()
    plt.show()
plot_mean()

In [ ]:
def boxplot_posdis(avg_path='average_distance.pkl', weak_path='weak_mwe_distance.pkl', strong_path='strong_mwe_distance.pkl'):
    avg_dist = pd.read_pickle(avg_path)
    weak_mwe_distance = pd.read_pickle(weak_path)
    strong_mwe_distance = pd.read_pickle(strong_path)
    
    avg_df = pd.DataFrame(avg_dist, columns = ['I', 'posdis', 'ignore']).drop(columns = ['ignore']).assign(Location='avg')
    weak_df = pd.DataFrame(weak_mwe_distance, columns = ['I', 'posdis', 'ignore']).drop(columns = ['ignore']).assign(Location='weak')
    weak_df = weak_df[weak_df['posdis']!=0].copy()
    strong_df = pd.DataFrame(strong_mwe_distance, columns = ['I', 'posdis', 'ignore']).drop(columns = ['ignore']).assign(Location='strong')
    cdf = pd.concat([avg_df, weak_df])#, strong_df])    
    cdf = cdf[cdf['posdis']<=5].copy()
    import seaborn as sns
    import matplotlib.pyplot as plt
    ax = sns.boxplot(x="posdis", y='I',hue="Location", data=cdf)     #hue="Letter",
    plt.show()

boxplot_posdis()

In [ ]:
# avg_df.boxplot(column = ['I'], by = 'posdis')

In [ ]:
# weak_df

In [ ]:
model(X[0:k])

In [ ]:


[x for x in testlist_text if 'Although he wrote' in x]


In [ ]:
pred_mode = 1 # [norm of softmax ]


# TODO: Fix the prediction fn. Might have to incorporate
#  target variable to get the logit for the target variable instead of max
predict_fn = llm_helper.get_prediction_fn(model, pred_mode = pred_mode)



torch.no_grad()

obj = RegressionGame(X = X[0:k], y=None, function = predict_fn, transform = torch.as_tensor)

X_samp = X[k:(N+k)]

shapley_values = np.empty((0, X.shape[1]))
partial_residuals = np.empty((0, X.shape[1]))
games = np.empty((0, 2 ** X.shape[1]))

print("SHape")
print(X.shape[1])

print("  ..ok!")
print("Generating explanations..")

for i in range(0, N):
    example_row = X_samp[i,:].reshape((1,X_samp.shape[1]))
    game = obj.getKernelSHAPGame(example_row)
    games = np.append(games, game.reshape((1,game.shape[0])), axis = 0)
    results, residualGame, origGame = getShapleyProjection(game)
    shapley_values = np.append(shapley_values,
                               np.array([np.flip(results[-1])]), axis=0)
    partial_residuals = np.append(partial_residuals,
                                  np.array([np.flip(norm(residualGame, axis = 0)/norm(origGame, axis = 0))]), axis = 0)
    print("%s/%s samples done." % (i+1, N))
    

    if i % 100 == 0:
        pd.DataFrame(X_samp).to_csv('data/llm_input.csv')
        pd.DataFrame(shapley_values).to_csv('data/llm_shapley_values.csv')
        pd.DataFrame(partial_residuals).to_csv('data/llm_partial_residuals.csv')

print(" Explanations saved to data/llm_*.csv!")

pd.DataFrame(X_samp).to_csv('data/llm_input.csv')
pd.DataFrame(shapley_values).to_csv('data/llm_shapley_values.csv')
pd.DataFrame(partial_residuals).to_csv('data/llm_partial_residuals.csv')

"""
TODO: 
1. How to get base line features for the text generation process? 
2. What is the appropriate metric for shapley score. 
"""

In [ ]:
partial_residuals

In [ ]:
# predict_fn = llm_helper.get_prediction_fn(model, pred_mode = 1)



# torch.no_grad()


# # obj = RegressionGame(X = X[0:k], y=y[0:k], function = predict_fn, transform = torch.as_tensor)
# obj = RegressionGame(X = X[0:k], y=None, function = predict_fn, transform = torch.as_tensor)

# X_samp = X[k:(N+k)]

# shapley_values = np.empty((0, X.shape[1]))
# partial_residuals = np.empty((0, X.shape[1]))
# games = np.empty((0, 2 ** X.shape[1]))

# print("SHape")
# print(X.shape[1])

# print("  ..ok!")
# print("Generating explanations..")

# for i in range(0, N):
#     example_row = X_samp[i,:].reshape((1,X_samp.shape[1]))
#     game = obj.getKernelSHAPGame(example_row)
#     games = np.append(games, game.reshape((1,game.shape[0])), axis = 0)
#     results, residualGame, origGame = getShapleyProjection(game)
#     shapley_values = np.append(shapley_values,
#                                np.array([np.flip(results[-1])]), axis=0)
#     partial_residuals = np.append(partial_residuals,
#                                   np.array([np.flip(norm(residualGame, axis = 0)/norm(origGame, axis = 0))]), axis = 0)
#     print("%s/%s samples done." % (i+1, N))

# print(" Explanations saved to data/llm_*.csv!")

# pd.DataFrame(X_samp).to_csv('data/llm_input.csv')
# pd.DataFrame(shapley_values).to_csv('data/llm_shapley_values.csv')
# pd.DataFrame(partial_residuals).to_csv('data/llm_partial_residuals.csv')

# """
# TODO: 
# 1. How to get base line features for the text generation process? 
# 2. What is the appropriate metric for shapley score. 
# 3. issue here, check why I was not able to make k = 40. model is probably not able to take more than 25. Check
# """

In [ ]:
shapley_values

In [ ]:
partial_residuals